In [1]:
import libriichi
help(libriichi)

Help on module libriichi:

NAME
    libriichi

DESCRIPTION
    This module provides implementations of the riichi mahjong including the
    following features:

    - The core feature - player state maintenance driven by mjai events (via
      `state.PlayerState`).
    - Read mjai logs and produce a batch of instances for training (via
      `dataset`).
    - Self-play under standard Tenhou rules (via `arena`).
    - Definitions of observation and action space for Mortal (via `consts`).
    - Statistical works on mjai logs (via `stat.Stat`).
    - mjai interface (via `mjai.Bot`).

DATA
    __all__ = ['__profile__', '__version__', 'consts', 'state', 'dataset',...
    __profile__ = 'release'

VERSION
    0.1.0

FILE
    e:\code\mortal\mortal\libriichi.pyd




In [2]:
loader = libriichi.dataset.GameplayLoader(version = 1)

In [139]:
import libriichi  # 假设你的 Rust 模块名为 libriichi
import logging

def load_and_process_gameplay(rust_loader, mjai_log_path):
    """
    加载并处理单个 mjai 日志文件的 Gameplay 数据。
    """
    try:
        with open(mjai_log_path, 'r', encoding='utf-8') as f:
            log_content = f.read()
        gameplays = rust_loader.load_log(log_content)
        logging.info(f"Successfully loaded {len(gameplays)} Gameplay instances from {mjai_log_path}")
        print(gameplays)
        return gameplays
    except Exception as e:
        logging.error(f"Error loading log {mjai_log_path}: {e}")
        return None


# 示例：初始化 Rust 的 GameplayLoader
loaders = []
for version in [1, 2, 3, 4]:
    loader = libriichi.dataset.GameplayLoader(version=version)
    loaders.append(loader)

# 示例：加载并处理一个转换后的 mjai 日志文件
mjai_log_path = "20230101.html/2023010100gm-00a9-0000-7fcb3f13.json"
gameplays_v = []
for loader in loaders:
    gameplays = load_and_process_gameplay(loader, mjai_log_path)
    if gameplays is not None:
        gameplays_v.append(gameplays)



[<builtins.Gameplay object at 0x0000020D16DBC1A0>, <builtins.Gameplay object at 0x0000020D17B91000>, <builtins.Gameplay object at 0x0000020D17B91170>, <builtins.Gameplay object at 0x0000020D17B912E0>]
[<builtins.Gameplay object at 0x0000020D17B91450>, <builtins.Gameplay object at 0x0000020D17B92870>, <builtins.Gameplay object at 0x0000020D17B91730>, <builtins.Gameplay object at 0x0000020D17B91A10>]
[<builtins.Gameplay object at 0x0000020D17B91B80>, <builtins.Gameplay object at 0x0000020D17B90D20>, <builtins.Gameplay object at 0x0000020D17B91CF0>, <builtins.Gameplay object at 0x0000020D17B91E60>]
[<builtins.Gameplay object at 0x0000020D17B90A40>, <builtins.Gameplay object at 0x0000020D17B90E90>, <builtins.Gameplay object at 0x0000020D17B91FD0>, <builtins.Gameplay object at 0x0000020D17B918A0>]


In [140]:
obss = []
for gameplays in gameplays_v:
    for gameplay in gameplays:
        obss.append(gameplay.take_obs())
gameplays = gameplays_v[0]
actions = []
for gameplay in gameplays:
    actions.append(gameplay.take_actions())

In [154]:
version = 1
player_id = 1
decision_points_idx = 39
# print("obs len:", len(obss[(version-1)*4+player_id]))
# print("手牌编码:\n", obss[(version-1)*4+player_id][decision_points_idx][0:7])
# print("点数编码:\n", obss[(version-1)*4+player_id][decision_points_idx][7:11])
# print("排名编码:\n", obss[(version-1)*4+player_id][decision_points_idx][11:15])
# print("局数编码:\n", obss[(version-1)*4+player_id][decision_points_idx][15:19])
# print("本场和供托编码:\n", obss[(version-1)*4+player_id][decision_points_idx][19:39])
# print("场风与自风编码:\n", obss[(version-1)*4+player_id][decision_points_idx][39:41])
print("dora指示牌编码:\n", obss[(version-1)*4+player_id][decision_points_idx][41:48])
print("牌河的编码以四个通道一组，分别表示杠牌，舍牌，红宝牌和普通dora牌")
print("自家牌河编码(前6):\n")
for i in range(6):
    print(obss[(version-1)*4+player_id][decision_points_idx][48+i*4:48+i*4+4])
    print("--------------------------------------------------------------------")

dora指示牌编码:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
牌河的编码以四个通道一组，分别表示杠牌，舍牌，红宝牌和普通dora牌
自家牌河编码(前6):

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0

In [97]:
print("actions length:", len(actions[0]))
print("player0:\n", actions[0])

actions length: 69
player0:
 [9, 29, 0, 8, 31, 15, 12, 45, 11, 20, 27, 45, 23, 17, 37, 3, 45, 28, 31, 27, 33, 2, 27, 19, 45, 20, 45, 3, 3, 18, 27, 30, 27, 33, 45, 10, 33, 32, 27, 26, 31, 28, 5, 29, 0, 45, 15, 20, 2, 18, 40, 3, 32, 15, 24, 0, 8, 31, 9, 18, 45, 20, 8, 45, 37, 24, 21, 24, 43]


In [ ]:
akochan_agent = libriichi.arena.OneVsThree.ako